In [1]:
from datetime import date, datetime

import pandas as pd
import pytz
from database import Database
from sklearn.pipeline import Pipeline
from sqlalchemy.dialects.postgresql import insert
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

local_tz = pytz.timezone("Asia/Dubai")

In [2]:
date_fr = date.today(local_tz).strftime("%m/%d/%Y %H:%M:%S")
date_to = datetime.now(local_tz).strftime("%m/%d/%Y %H:%M:%S")

TypeError: date.today() takes no arguments (1 given)

In [ ]:
db = Database()

reservation = db.gwg_reservation

In [ ]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        ("data_encoder", BookingDataEncoder()),
    ]
)

In [ ]:
dfs = []

for destination in ["DU", "OM"]:
    booking_data = BookingData(destination, date_fr, date_to, 3).get()

    df = pipeline.fit_transform(booking_data)
    dfs.append(df)

In [ ]:
records = pd.concat(dfs, axis=0, ignore_index=True)

if records is not None and not records.empty:
    chunk_size = 1000
    for i in range(0, len(records), chunk_size):
        chunk = records.iloc[i : i + chunk_size]

        stmt = insert(reservation).values(chunk.to_dict(orient="records"))
        stmt = stmt.on_conflict_do_update(
            constraint="gwg_reservation_pkey", set_=dict(stmt.excluded)
        )

        with db.engine.begin() as connection:
            connection.execute(stmt)